## Test time inference using trained model for Multiple_points_singlenet_pred_xyz_Set_loss

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split,Subset
from torch.nn.parameter import Parameter
from logger import Logger
import os
from utils_twopoints import *
from visualize import show2D_2objs, show2D_3objs
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
torch.backends.cudnn.deterministic=True

plt.rcParams.update({'font.size': 15,'lines.markersize':10})

In [ ]:
np.random.seed(190)
torch.manual_seed(120) ##set the random seed so when construct the 

#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_21 CCD XYZ sweep')
#data_path='FS_LD9by9_All_CCD sweep xyz_avg_win20_normalizedbymax'

os.chdir('/home/zyhuang/WD/Net3D/2018_11_21 CCD XYZ sweep/')
data_path = 'FS_9by9_CMOS_avg_win20_normalizedbymax'

scale_z=True #divide z coord by 33.3 to make it has same range as x,y
lr=1e-3
num_epoch=2000
bs_test=282 #specify test set size
bs_train=100#remaining used as trainset
num_features=64
#input_dim=4*4*2
input_dim=9*9*2
nx,ny,nz=11,11,11

#relative_pos_list=[[(0,0,0),(1,2,3)]] # for two objs, single shift
#relative_pos_list=[[(0,0,1),(0,3,0)],[(0,1,3),(2,1,1)]] # for two objs, two shift
#relative_pos_list=[[(0,0,1),(0,3,0)],[(0,1,3),(2,1,1)],[(1,3,2),(0,1,1)]] #for two objs, three shifts
#relative_pos_list=[[(0,0,1),(0,3,0),(1,0,2)],[(0,1,3),(2,1,1),(3,3,2)]] # for three objs, two shift
relative_pos_list=[[(0,0,1),(0,3,0),(1,0,2)],[(0,1,3),(2,1,1),(3,3,2)],[(3,2,2),(0,2,1),(3,0,2)]] # for three objs, three shift
Nshift=len(relative_pos_list)
Nobj=len(relative_pos_list[0])
Nobj_eng={'2':'two','3':'three'}

if Nshift==1:
    relative_pos=relative_pos_list[0]
    ds_Full=globals()['FSdataset_'+Nobj_eng[str(Nobj)]+'points_v2'](data_path,relative_pos=relative_pos,nx=nx,ny=ny,nz=nz,scale_z=scale_z)
    save_name=str(Nobj)+'_objs_singlenetxyz_Set_loss_1shift_scale_z_'+str(scale_z)+'/'+"".join([s for s in str(relative_pos) if s.isdigit()])+'/depthnet4_Nobj_xyz('+str(num_features)+'features)'\
    +'_adam_lr_'+str(lr)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)
else:
    ds_Full=globals()['FSdataset_'+Nobj_eng[str(Nobj)]+'points_multipleshifts_v2'](data_path,relative_pos_list=relative_pos_list,nx=nx,ny=ny,nz=nz,scale_z=scale_z)
    save_name=str(Nobj)+'_objs_singlenetxyz_Set_loss_'+str(Nshift)+'shifts_scale_z_'+str(scale_z)+'/'+"".join([s for s in str(relative_pos_list) if s.isdigit()])\
    +'/depthnet4_Nobj_xyz('+str(num_features)+'features)'+'_adam_lr_'+str(lr)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)
print("Length of Full dataset is %s" %len(ds_Full))
ds_train,ds_test=random_split(ds_Full, [len(ds_Full)-bs_test,bs_test])   
train_loader=DataLoader(ds_train, batch_size=bs_train,shuffle=True)
test_loader=DataLoader(ds_test, batch_size=bs_test,shuffle=False)

#dtype = torch.FloatTensor ##run on cpu
dtype = torch.cuda.FloatTensor 

net=depthnet4_Nobj_xyz(input_dim,num_features,Nobj=Nobj)
net.type(dtype)
net.load_state_dict(torch.load('logs_paper_data/model.pth'))


In [ ]:
net.eval()
test_bat=next(iter(test_loader))
FS_bat,trueXYZobjs_bat=test_bat['FS'],test_bat['xyz'][:,:]
#predXYZobjs_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()[:,[0,2,4,1,3,5]]
predXYZobjs_bat=net(Variable(FS_bat).type(dtype)).data.cpu()

#scale back the z coordinates
#predXYZobjs_bat[:,[2,5]] = predXYZobjs_bat[:,[2,5]]*33.3
#trueXYZobjs_bat[:,[2,5]] = trueXYZobjs_bat[:,[2,5]]*33.3
predXYZobjs_bat[:,[2,5,8]] = predXYZobjs_bat[:,[2,5,8]]*33.3
trueXYZobjs_bat[:,[2,5,8]] = trueXYZobjs_bat[:,[2,5,8]]*33.3

print(predXYZobjs_bat)
print(trueXYZobjs_bat)

offset = 215 # how many offset in the sample index to make the plot
globals()['show2D_'+str(Nobj)+'objs'](trueXYZobjs_bat,predXYZobjs_bat,howmany=4,offset=offset,save_name='logs_paper_data/2Ds-offset_'+str(offset))   

In [ ]:
##Timing benchmark code
# res=32
# input_dim=res*res*2
# net=depthnet4_Nobj_xyz(input_dim,num_features,Nobj=3)
# net.type(dtype)
# net.eval()

# FS_bat=torch.zeros(1,2,res,res)
# #predXYZobjs_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()[:,[0,2,4,1,3,5]]

# times = []
# for _ in range(5000):
#     start = torch.cuda.Event(enable_timing=True)
#     end = torch.cuda.Event(enable_timing=True)
#     start.record()
#     with torch.no_grad():
#         predXYZobjs_bat=net(Variable(FS_bat).type(dtype))
#         end.record()
#         torch.cuda.synchronize()
#         #print(start.elapsed_time(end))
#         times.append(start.elapsed_time(end))
# print('Mean is {0:.3f}'.format(np.mean(times)))
